# Unsupervised methods

In this lesson, we'll cover unsupervised computational text anlalysis approaches. The central methods covered are TF-IDF and Topic Modeling. Both of these are common approachs in the social sciences and humanities.

[DTM/TF-IDF](#dtm)<br>

[Topic modeling](#topics)<br>

### Today you will
* Understand the DTM and why it's important to text analysis
* Learn how to create a DTM in Python
* Learn basic functionality of Python's package scikit-learn
* Understand tf-idf scores
* Learn a simple way to identify distinctive words
* Implement a basic topic modeling algorithm and learn how to tweak it
* In the process, gain more familiarity and comfort with the Pandas package and manipulating data


### Key Jargon
* *Document Term Matrix*:
    * a matrix that describes the frequency of terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.
* *TF-IDF Scores*: 
    * short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
* *Topic Modeling*:
    * A general class of statistical models that uncover abstract topics within a text. It uses the co-occurrence of words within documents, compared to their distribution across documents, to uncover these abstract themes. The output is a list of weighted words, which indicate the subject of each topic, and a weight distribution across topics for each document.
    
* *LDA*:
    * Latent Dirichlet Allocation. A particular model for topic modeling. It does not take document order into account, unlike other topic modeling algorithms.

## DTM/TF-IDF <a id='dtm'></a>

In this lesson we will use Python's scikit-learn package learn to make a document term matrix from a .csv Music Reviews dataset (collected from MetaCritic.com). We will then use the DTM and a word weighting technique called tf-idf (term frequency inverse document frequency) to identify important and discriminating words within this dataset (utilizing the Pandas package). The illustrating question: **what words distinguish reviews of Rap albums, Indie Rock albums, and Jazz albums?**

In [1]:
import os
import numpy as np
import pandas as pd

DATA_DIR = 'data'
music_fname = 'music_reviews.csv'
music_fname = os.path.join(DATA_DIR, music_fname)

### First attempt at reading in file

In [3]:
reviews = pd.read_csv(music_fname, sep='\t')
reviews.head()

,album,artist,genre,release_date,critic,score,body
0,Don't Panic,All Time Low,Pop/Rock,2012-10-09 00:00:00,Kerrang!,74.0,While For Baltimore proves they can still writ...
1,Fear and Saturday Night,Ryan Bingham,Country,2015-01-20 00:00:00,Uncut,70.0,There's nothing fake about the purgatorial nar...
2,The Way I'm Livin',Lee Ann Womack,Country,2014-09-23 00:00:00,Q Magazine,84.0,All life's disastrous lows are here on a caree...
3,Doris,Earl Sweatshirt,Rap,2013-08-20 00:00:00,Pitchfork,82.0,"With Doris, Odd Future’s Odysseus is finally b..."
4,Giraffe,Echoboy,Rock,2003-02-25 00:00:00,AllMusic,71.0,Though Giraffe is definitely Echoboy's most im...


Print the text of the first review.

In [4]:
print(reviews['body'][0])

While For Baltimore proves they can still write a grade A banger when they put their mind to it, too many songs are destined to have "must try harder" stamped on their report card. [13 Oct 2012, p.52]


### Explore the Data using Pandas

Let's first look at some descriptive statistics about this dataset, to get a feel for what's in it. We'll do this using the Pandas package. 

Note: this is always good practice. It serves two purposes. It checks to make sure your data is correct, and there's no major errors. It also keeps you in touch with your data, which will help with interpretation. <3 your data!

First, what genres are in this dataset, and how many reviews in each genre?

In [5]:
#We can count this using the value_counts() function
reviews['genre'].value_counts()

Pop/Rock                  1486
Indie                     1115
Rock                       932
Electronic                 513
Rap                        363
Pop                        149
Country                    140
R&B;                       112
Folk                        70
Alternative/Indie Rock      42
Dance                       41
Jazz                        38
Name: genre, dtype: int64

The first thing most people do is to `describe` their data. (This is the `summary` command in R, or the `sum` command in Stata).

In [6]:
#There's only one numeric column in our data so we only get one column for output.
reviews.describe()

,score
count,5001.000000
mean,72.684223
std,8.714896
min,7.400000
25%,68.000000
50%,74.000000
75%,79.000000
max,100.000000


This only gets us numerical summaries. To get summaries of some of the other columns, we can explicitly ask for it.

In [8]:
reviews.describe(include=['O'])

,album,artist,genre,release_date,critic,body
count,5001,5001,5001,5001,5001,5001
unique,3799,2607,12,956,592,4998
top,13,Various Artists,Pop/Rock,2011-09-13 00:00:00,AllMusic,A welcome surprise.
freq,5,22,1486,29,282,2


Who were the reviewers?

In [9]:
reviews['critic'].value_counts().head(10)

AllMusic                     282
PopMatters                   228
Pitchfork                    207
Q Magazine                   178
Uncut                        171
Mojo                         137
Drowned In Sound             132
New Musical Express (NME)    127
The A.V. Club                121
Rolling Stone                112
Name: critic, dtype: int64

And the artists?

In [10]:
reviews['artist'].value_counts().head(10)

Various Artists      22
R.E.M.               16
Arcade Fire          14
Sigur Rós            13
Belle & Sebastian    12
Brian Eno            11
Weezer               10
LCD Soundsystem      10
The Raveonettes      10
Mogwai               10
Name: artist, dtype: int64

We can get the average score as follows:

In [11]:
reviews['score'].mean()

72.68422315536893

Now we want to know the average score for each genre? To do this, we use Pandas `groupby` function. You'll want to get very familiar with the `groupby` function. It's quite powerful. (Similar to `collapse` on Stata)

In [12]:
reviews_grouped_by_genre = reviews.groupby("genre")
reviews_grouped_by_genre['score'].mean().sort_values(ascending=False)

genre
Jazz                      77.631579
Folk                      75.900000
Indie                     74.400897
Country                   74.071429
Alternative/Indie Rock    73.928571
Electronic                73.140351
Pop/Rock                  73.033782
R&B;                      72.366071
Rap                       72.173554
Rock                      70.754292
Dance                     70.146341
Pop                       64.608054
Name: score, dtype: float64

### Creating the DTM using scikit-learn

Ok, that's the summary of the metadata. Next, we turn to analyzing the text of the reviews. Remember, the text is stored in the 'body' column. First, a preprocessing step to remove numbers.

In [17]:
def remove_digits(comment):
    return ''.join([ch for ch in comment if not ch.isdigit()])

reviews['body_without_digits'] = reviews['body'].apply(remove_digits)


In [18]:
reviews['body_without_digits'].head()

0    While For Baltimore proves they can still writ...
1    There's nothing fake about the purgatorial nar...
2    All life's disastrous lows are here on a caree...
3    With Doris, Odd Future’s Odysseus is finally b...
4    Though Giraffe is definitely Echoboy's most im...
Name: body_without_digits, dtype: object

### CountVectorizer Function

Our next step is to turn the text into a document term matrix using the scikit-learn function called `CountVectorizer`.

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer()
sparse_dtm = countvec.fit_transform(reviews['body_without_digits'])

Great! We made a DTM! Let's look at it.

In [20]:
sparse_dtm

<5001x16139 sparse matrix of type '<class 'numpy.int64'>'
	with 124340 stored elements in Compressed Sparse Row format>

This format is called Compressed Sparse Format. It save a lot of memory to store the dtm in this format, but it is difficult to look at for a human. To illustrate the techniques in this lesson we will first convert this matrix back to a Pandas DataFrame, a format we're more familiar with. For larger datasets, you will have to use the Compressed Sparse Format. Putting it into a DataFrame, however, will enable us to get more comfortable with Pandas!

In [21]:
dtm = pd.DataFrame(sparse_dtm.toarray(), columns=countvec.get_feature_names(), index=reviews.index)
dtm

,aa,aaaa,aahs,aaliyah,aaron,ab,abandon,abandoned,abandoning,abc,...,zone,zones,zoo,zooey,zoomer,zu,zydeco,álbum,être,über
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### What can we do with a DTM?

We can quickly identify the most frequent words

In [22]:
dtm.sum().sort_values(ascending=False).head(10)

the      7406
and      4557
of       4400
to       3175
is       2914
it       2608
that     2039
in       1775
album    1719
this     1518
dtype: int64

### Challenge

* Print out the most infrequent words rather than the most frequent words. You can look at the [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats) for more information.
* Print the average number of times each word is used in a review.
* Print this out sorted from highest to lowest.

### TF-IDF scores

How to find distinctive words in a corpus is a long-standing question in text analysis. Today, we'll learn one simple approach to this: TF-IDF. The idea behind words scores is to weight words not just by their frequency, but by their frequency in one document compared to their distribution across all documents. Words that are frequent, but are also used in every single document, will not be distinguising. We want to identify words that are unevenly distributed across the corpus.

One of the most popular ways to weight words (beyond frequency counts) is `tf-idf score`. By offsetting the frequency of a word by its document frequency (the number of documents in which it appears) will in theory filter out common terms such as 'the', 'of', and 'and'.

Traditionally, the inverse document frequency is calculated as such:

number_of_documents / number_documents_with_term

so:

tfidf_word1 = word1_frequency_document1 * (number_of_documents / number_document_with_word1)

You can, and often should, normalize the numerator: 

tfidf_word1 = (word1_frequency_document1 / word_count_document1) * (number_of_documents / number_document_with_word1)

We can calculate this manually, but scikit-learn has a built-in function to do so. This function also uses log frequencies, so the numbers will not correspond excactly to the calculations above. We'll use the [scikit-learn calculation](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), but a challenge for you: use Pandas to calculate this manually. 

### TF-IDFVectorizer Function

To do so, we simply do the same thing we did above with CountVectorizer, but instead we use the function TfidfVectorizer.

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvec = TfidfVectorizer()
sparse_tfidf = tfidfvec.fit_transform(reviews['body_without_digits'])
sparse_tfidf

<5001x16139 sparse matrix of type '<class 'numpy.float64'>'
	with 124340 stored elements in Compressed Sparse Row format>

In [24]:
tfidf = pd.DataFrame(sparse_tfidf.toarray(), columns=tfidfvec.get_feature_names(), index=reviews.index)
tfidf.head()

,aa,aaaa,aahs,aaliyah,aaron,ab,abandon,abandoned,abandoning,abc,...,zone,zones,zoo,zooey,zoomer,zu,zydeco,álbum,être,über
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's look at the 20 words with highest tf-idf weights.

In [25]:
tfidf.max().sort_values(ascending=False).head(20)

brill         1.000000
perfect       1.000000
yummy         1.000000
pppperfect    1.000000
awesome       1.000000
wonderfull    1.000000
meh           1.000000
stars         1.000000
subpar        0.959257
ga            0.908259
masterful     0.898620
grower        0.888624
likable       0.867803
acirc         0.867003
great         0.864253
infectious    0.859996
blank         0.854475
thrilling     0.848810
smart         0.847852
stuff         0.834479
dtype: float64

Ok! We have successfully identified content words, without removing stop words.

### Identifying Distinctive Words

What can we do with this? These scores are best used when you want to identify distinctive words for individual documents, or groups of documents, compared to other groups or the corpus as a whole. To illustrate this, let's compare three genres and identify the most distinctive words by genre.

First we add in a column of genre.

In [26]:
tfidf['genre_'] = reviews['genre']
tfidf.head()

,aa,aaaa,aahs,aaliyah,aaron,ab,abandon,abandoned,abandoning,abc,...,zones,zoo,zooey,zoomer,zu,zydeco,álbum,être,über,genre_
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pop/Rock
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Country
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Country
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Rap
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Rock


Now lets compare the words with the highest tf-idf weight for each genre. 

In [27]:
rap = tfidf[tfidf['genre_']=='Rap']
indie = tfidf[tfidf['genre_']=='Indie']
jazz = tfidf[tfidf['genre_']=='Jazz']

rap.max(numeric_only=True).sort_values(ascending=False).head()

blank        0.854475
waste        0.755918
amiable      0.730963
awesomely    0.717079
joyless      0.687687
dtype: float64

In [28]:
indie.max(numeric_only=True).sort_values(ascending=False).head()

meh           1.0
awesome       1.0
wonderfull    1.0
perfect       1.0
yummy         1.0
dtype: float64

In [29]:
jazz.max(numeric_only=True).sort_values(ascending=False).head()

purely        0.544477
descending    0.519218
devotional    0.507724
recordings    0.499963
languid       0.487715
dtype: float64

There we go! A method of identifying distinctive words.

### Challenge

Instead of outputting the highest weighted words, output the lowest weighted words. How should we interpret these words?

## Topic modeling <a id='topics'></a>

The goal of topic models can be twofold: 1/ learning something about the topics themselves, i.e what the the ext is about 2/ reduce the dimensionality of text to represent a document as a weighted average of K topics instead of a vector of token counts over the whole vocabulary. In the latter case, topic modeling a way to treat text as any data in a more tractable way for any subsequent statistical analysis (linear/logistic regression, etc). 

There are many topic modeling algorithms, but we'll use LDA. This is a standard model to use. Again, the goal is not to learn everything you need to know about topic modeling. Instead, this will provide you some starter code to run a simple model, with the idea that you can use this base of knowledge to explore this further.

We will run Latent Dirichlet Allocation, the most basic and the oldest version of topic modeling$^1$. We will run this in one big chunk of code. Our challenge: use our knowledge of scikit-learn that we gained above to walk through the code to understand what it is doing. Your challenge: figure out how to modify this code to work on your own data, and/or tweak the parameters to get better output.

First, a bit of theory. LDA is a generative model - a model over the entire data generating process - in which a document is a mixture of topics and topics are probability distributions over tokens in the vocabulary. The (normalized) frequency of word $j$ in document $i$ can be written as:
$q_{ij} = v_{i1}*\theta_{1j} + v_{i2}*\theta_{2j} + ... + v_{iK}*\theta_{Kj}$
where K is the total number of topics, $\theta_{kj}$ is the probability that word $j$ shows up in topic $k$ and $v_{ik}$ is the weight assigned to topic $k$ in document $i$. The model treats $v$ and $\theta$ as generated from Dirichlet-distributed priors and can be estimated through Maximum Likelihood or Bayesian methods.

Note: we will be using a different dataset for this technique. The music reviews in the above dataset are often short, one word or one sentence reviews. Topic modeling is not really appropriate for texts that are this short. Instead, we want texts that are longer and are composed of multiple topics each. For this exercise we will use a database of children's literature from the 19th century. 

The data were compiled by students in this course: http://english197s2015.pbworks.com/w/page/93127947/FrontPage
Found here: http://dhresourcesforprojectbuilding.pbworks.com/w/page/69244469/Data%20Collections%20and%20Datasets#demo-corpora

That page has additional corpora, for those interested in exploring text analysis further.

$^1$ Reference: Blei, D. M., A. Y. Ng, and M. I. Jordan (2003). Latent Dirichlet allocation. Journal of Machine
Learning Research 3, 993–1022.

In [30]:
literature_fname = os.path.join(DATA_DIR, 'childrens_lit.csv.bz2')
df_lit = pd.read_csv(literature_fname, sep='\t', encoding = 'utf-8', compression = 'bz2', index_col=0)

#drop rows where the text is missing
df_lit = df_lit.dropna(subset=['text'])
df_lit.head()

,title,author gender,year,text
0,A Dog with a Bad Name,Male,1886,A DOG WITH A BAD NAME BY TALBOT BAINES REED ...
1,A Final Reckoning,Male,1887,A Final Reckoning: A Tale of Bush Life in Aust...
2,"A House Party, Don Gesualdo, and A Rainy June",Female,1887,A HOUSE-PARTY Don Gesualdo and A Rainy June...
3,A Houseful of Girls,Female,1889,"A HOUSEFUL OF GIRLS. BY SARAH TYTLER, AUTHOR ..."
4,A Little Country Girl,Female,1885,"LITTLE COUNTRY GIRL. BY SUSAN COOLIDGE, ..."


Now we're ready to fit the model. This requires the use of CountVectorizer, which we've already used, and the scikit-learn function LatentDirichletAllocation.

See [here](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) for more information about this function. 

First, we have to import it from sklearn.

In [31]:
from sklearn.decomposition import LatentDirichletAllocation

In sklearn, the input to LDA is a DTM (with either counts or TF-IDF scores).

In [32]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.80, min_df=50,
                                   max_features=5000,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df_lit['text'])

In [33]:
tf_vectorizer = CountVectorizer(max_df=0.80, min_df=50,
                                max_features=5000,
                                stop_words='english'
                                )
tf = tf_vectorizer.fit_transform(df_lit['text'])

This is where we fit the model.

In [34]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
lda = LatentDirichletAllocation(n_topics=5, max_iter=20, random_state=0)
lda = lda.fit(tf)

This is a function to print out the top words for each topic in a pretty way. Don't worry too much about understanding every line of this code.

In [35]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #{}:".format(topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [36]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 50)


Topic #0:
doctor girls papa mamma sister baby street aunt london dr tom sweet tea study presently flower cousin darling wasn everybody project loved nurse ladies office stairs shop ain class carriage doesn ma remarked bell flowers mary silk lovely term st gray vi christmas sisters purse em anybody garden rooms pounds

Topic #1:
dick uncle doctor jack er tom ain den yer fish em lads rock rope gun wolf birds beneath stream ay shock ha rocks mate tail eh moments garden excitedly sand thrust fishing chap nay ye softly leg penny ship tremendous gazing ashore bird angrily hook farther gray task jump growled

Topic #2:
king camp prince indian queen tom mountain snow ice ha hut rock arrows band palace forest village savage observed ship rode island gun youth shore maiden mountains giant ye rocks em stream presently dogs ride majesty animal bushes huge animals remarked hunting city valley native knife meat hunt birds pine

Topic #3:
project ye works deck ship shore foundation george vessel isl

One thing we may want to do with the output is compare the prevalence of each topic across documents. A simple way to do this (but not memory efficient), is to merge the topic distribution back into the Pandas dataframe.

First get the topic distribution array.

In [37]:
topic_dist = lda.transform(tf)
topic_dist

array([[8.50969215e-01, 1.88397896e-02, 7.92713226e-02, 2.59696048e-05,
        5.08937029e-02],
       [1.74760423e-01, 1.35098342e-01, 1.88079200e-02, 1.46784348e-01,
        5.24548967e-01],
       [8.79249569e-01, 2.80727250e-05, 1.13942765e-01, 1.86976637e-03,
        4.90982655e-03],
       [9.96054085e-01, 2.36876490e-05, 2.37652924e-05, 1.26746315e-04,
        3.77171592e-03],
       [8.83320163e-01, 5.94319337e-02, 2.14224095e-02, 3.00354503e-02,
        5.79004306e-03],
       [7.85930550e-01, 7.14902357e-02, 1.42484763e-01, 4.72674394e-05,
        4.71837759e-05],
       [4.43253938e-01, 5.52741806e-01, 1.05422316e-04, 3.79367377e-03,
        1.05160134e-04],
       [9.91130478e-01, 2.79129951e-05, 6.82648996e-04, 2.79847538e-05,
        8.13097523e-03],
       [1.20478669e-04, 1.77301661e-02, 9.19847410e-01, 6.22665502e-02,
        3.53948468e-05],
       [5.42320701e-05, 1.71155175e-01, 8.27152641e-01, 5.43966180e-05,
        1.58355447e-03],
       [6.22429394e-05, 6.2126

Merge back with original dataframe

In [38]:
topic_dist_df = pd.DataFrame(topic_dist)
df_w_topics = topic_dist_df.join(df_lit)
df_w_topics

,0,1,2,3,4,title,author gender,year,text
0,0.850969,0.018840,0.079271,0.000026,0.050894,A Dog with a Bad Name,Male,1886.0,A DOG WITH A BAD NAME BY TALBOT BAINES REED ...
1,0.174760,0.135098,0.018808,0.146784,0.524549,A Final Reckoning,Male,1887.0,A Final Reckoning: A Tale of Bush Life in Aust...
2,0.879250,0.000028,0.113943,0.001870,0.004910,"A House Party, Don Gesualdo, and A Rainy June",Female,1887.0,A HOUSE-PARTY Don Gesualdo and A Rainy June...
3,0.996054,0.000024,0.000024,0.000127,0.003772,A Houseful of Girls,Female,1889.0,"A HOUSEFUL OF GIRLS. BY SARAH TYTLER, AUTHOR ..."
4,0.883320,0.059432,0.021422,0.030035,0.005790,A Little Country Girl,Female,1885.0,"LITTLE COUNTRY GIRL. BY SUSAN COOLIDGE, ..."
5,0.785931,0.071490,0.142485,0.000047,0.000047,A Round Dozen,Female,1883.0,\n A ROUND DOZEN. [Illustration: TOINETTE AND...
6,0.443254,0.552742,0.000105,0.003794,0.000105,A Sailor's Lass,Female,1886.0,"A SAILOR'S LASS by EMMA LESLIE, Author of ""..."
7,0.991130,0.000028,0.000683,0.000028,0.008131,A World of Girls,Female,1886.0,A WORLD OF GIRLS: THE STORY OF A SCHOOL. By ...
8,0.000120,0.017730,0.919847,0.062267,0.000035,Adrift in the Wild,Male,1887.0,Adrift in the Wilds; ...
9,0.000054,0.171155,0.827153,0.000054,0.001584,Adventures in Africa,Male,1883.0,"ADVENTURES IN AFRICA, BY W.H.G. KINGSTON. C..."


Now we can chech the average weight of each topic across gender using `groupby`.

In [39]:
grouped = df_w_topics.groupby('author gender')
grouped[0].mean().sort_values(ascending=False)

author gender
Female    0.565934
Male      0.192484
Name: 0, dtype: float64

## Challenge

Modify the script above to:
* increase the number of topics
* increase the number of printed top words per topic
* fit the model to the tf-idf matrix instead of the tf one

In [40]:
### increase number of topics
import itertools
num_topics_ls = [3, 5, 10, 20, 50]
num_topwords_ls = [20, 50, 100]
use_tf_ls = [True, False]
for n_topics, n_topwords, use_tf in itertools.product(num_topics_ls, num_topwords_ls, use_tf_ls):
    print("Topics: {}; Top Words: {}; tf: {}".format(n_topics, n_topwords, use_tf))
    lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=20, random_state=0)
    if use_tf == True:
        lda = lda.fit(tf)
    else:
        lda = lda.fit(tfidf)
    tf_feature_names = tf_vectorizer.get_feature_names()
    print_top_words(lda, tf_feature_names, n_topwords)

# ### increase number of printed top words per topic
# tf_feature_names = tf_vectorizer.get_feature_names()
# print_top_words(lda, tf_feature_names, 50)

Topics: 3; Top Words: 20; tf: True

Topic #0:
project doctor girls sister papa mamma works london street baby sweet dr tom remarked tea ye foundation aunt youth office

Topic #1:
dick uncle jack doctor er ain tom yer den fish em rock lads gun rope wolf ye ha ay rocks

Topic #2:
king army french troops camp attack ship officers prince john soldiers officer city village guns frank shore rode regiment march

Topics: 3; Top Words: 20; tf: False

Topic #0:
project papa mamma doctor girls ma aunt dr cousin gray sister dick tea foundation baby london street sweet darling vi

Topic #1:
dick er uncle fish ain guns yer den tail project doctor jack sunk distinguished charming drifted suited powder attempted consisting

Topic #2:
project doctor king dick uncle tom camp ship jack girls prince troops army shore ye french em deck yer john

Topics: 3; Top Words: 50; tf: True

Topic #0:
project doctor girls sister papa mamma works london street baby sweet dr tom remarked tea ye foundation aunt youth of


Topic #0:
doctor girls papa mamma sister baby street aunt london dr tom sweet tea study presently flower cousin darling wasn everybody project loved nurse ladies office stairs shop ain class carriage doesn ma remarked bell flowers mary silk lovely term st gray vi christmas sisters purse em anybody garden rooms pounds

Topic #1:
dick uncle doctor jack er tom ain den yer fish em lads rock rope gun wolf birds beneath stream ay shock ha rocks mate tail eh moments garden excitedly sand thrust fishing chap nay ye softly leg penny ship tremendous gazing ashore bird angrily hook farther gray task jump growled

Topic #2:
king camp prince indian queen tom mountain snow ice ha hut rock arrows band palace forest village savage observed ship rode island gun youth shore maiden mountains giant ye rocks em stream presently dogs ride majesty animal bushes huge animals remarked hunting city valley native knife meat hunt birds pine

Topic #3:
project ye works deck ship shore foundation george vessel isl


Topic #0:
doctor girls papa mamma sister baby street aunt london sweet project tom dr tea study presently flower darling office everybody

Topic #1:
dick doctor uncle tom jack fish em lads rope rock birds shock beneath ay gun stream garden excitedly moments fishing

Topic #2:
french troops officers army attack guns officer tom soldiers regiment british camp village ship march jack fort wounded column native

Topic #3:
project doctor church ma mary gray girls thou works soldier regiment james rode public village officer george st cousin soldiers

Topic #4:
frank james king shore lake camp village forest boats coast attack troops french native fort woods stream ship army guns

Topic #5:
project ye works ship george foundation island shore em observed youth ice deck ay agreement vessel remarked crew fish considerable

Topic #6:
er uncle jack ain den yer wolf folks lion tail gun dish sing jump aunt study seed doctor bag goodness

Topic #7:
deck frank shore ship vessel boats island cabin s


Topic #0:
doctor girls papa mamma sister baby street aunt london sweet project tom dr tea study presently flower darling office everybody wasn ladies remarked nurse loved stairs cousin shop carriage ma lovely flowers class doesn term bell vi silk sisters rooms christmas ain em youth jolly anybody garden queer fifth precious pounds train dig hadn public guess match purse miserable kiss demanded temper works delightful grace lessons uncle kitchen st awfully eh tender reader writing john court sympathy cheek regard city desk married ball bye gentlemen field shillings race village picture music forgive softly crying tiny cheerful loving mary french maid

Topic #1:
dick doctor uncle tom jack fish em lads rope rock birds shock beneath ay gun stream garden excitedly moments fishing sand mate thrust nay eh rocks ha softly penny ye hook angrily gazing fruit ashore tremendous chap task growled leg gray ship surface farther shore stones roar horrible yer stared ladder basket bird ain tail pole k


Topic #0:
george office london street deck shop bones lake uncle shore demanded regard st passengers island cabin situation vessel ladies tom

Topic #1:
dick uncle fish rope em shock rock garden fishing hook ye ladder ay sand eh basket lads rocks yer works

Topic #2:
project works ship ice island foundation observed queen shore youth remarked snow vessel agreement crew deck considerable fish ye em

Topic #3:
gray project girls church cousin james st dr college everybody works doctor aunt teacher anybody public street married cheese ladies

Topic #4:
uncle jack king frank dick john ship doctor deck french troops shore city attack camp boats street village tom prince

Topic #5:
sunday cousin shop william london ma girls situation purse class mary teacher tea bible bag servant uncle rule foolish street

Topic #6:
doctor ain james dr em yer project papa street john garden city slave fish darling sister er patient london pat

Topic #7:
frank bush ain camp dick village pounds em uncle stati

...


Topic #0:
project doctor london dr papa dick street girls office mamma tom remarked sister gray island cousin tea sweet bones foundation

Topic #1:
dick fish tom doctor guns prince king rope lads project shore french attack uncle troops rock army camp shock ship

Topic #2:
ice tom french king doctor project snow prince indian camp officers em jack village attack ship stream army soldiers guns

Topic #3:
gray cousin girls aunt church mamma project everybody dr baby folks anybody papa marry tea wasn married somebody george st

Topic #4:
king jack frank prince ship troops john attack city uncle french soldiers village doctor camp shore deck army boats street

Topic #5:
king girls doctor project ship shore island army city rock street ye camp prince tom john london deck sister gray

Topic #6:
doctor john project city em girls ain camp troops attack king mountain london baby army rock sister dick indian fish

Topic #7:
king dick prince frank shore attack tom village girls ship john soldier


Topic #0:
project doctor london dr papa dick street girls office mamma tom remarked sister gray island cousin tea sweet bones foundation ladies ma james ship baby observed works queen french ha stairs court attack youth village public shore officers king lake army loved em states presently flowers camp darling cat agreement

Topic #1:
dick fish tom doctor guns prince king rope lads project shore french attack uncle troops rock army camp shock ship jack ay native officers island tail rocks girls city works officer fort british ha boats village court gentlemen pole beneath garden eh tremendous basket ice gun queen stream london advanced

Topic #2:
ice tom french king doctor project snow prince indian camp officers em jack village attack ship stream army soldiers guns deck hut troops shore queen officer mountain boats girls lads british gun fort island works sister band considerable tremendous rope sailor ye london vessel south dogs mass observed youth frank

Topic #3:
gray cousin girls 


Topic #0:
mass dick mail folks seating shone moonlight doctor minded discussed select shouldn beauty believing songs wont plunged dreary entertainment wounds loved sufficed subdued rejoiced desired calculate veins delicate brush undertaking papa pipe riding reported absorbed cliffs masses wicked furiously ordering theirs sack ascertain cheap appeal sheet animal charged sobbing thoughtfully

Topic #1:
sunk distinguished drifted fish suited guns consisting charming tail subsided peril attempted project unusual powder invalid plate blame prize stunned granted respected lamp amazement larger solid facing bleeding reported reputation stock clapping scratched meat profit bade string supposing climbing roar pour awe absurd tumbled thread tie tower stolen keen honestly

Topic #2:
succeed weeping abode descended ice inquiry flowers friendship brain proceeding bodies sack answering imagined kicking wooden portions accounts shop burnt trip blankets rage limited trade stream fruit shattered retir


Topic #0:
project doctor london dr papa dick street girls office mamma tom remarked sister gray island cousin tea sweet bones foundation ladies ma james ship baby observed works queen french ha stairs court attack youth village public shore officers king lake army loved em states presently flowers camp darling cat agreement station south john city ain carriage nurse lamp wasn aunt everybody arrival fort gentlemen considerable train christmas shop ye rooms pounds reader united uncle anybody deck mail private officer field copy doesn demanded bell boats hut beauty study rope married ride job regard sisters mass guess queer soldiers st vi

Topic #1:
dick fish tom doctor guns prince king rope lads project shore french attack uncle troops rock army camp shock ship jack ay native officers island tail rocks girls city works officer fort british ha boats village court gentlemen pole beneath garden eh tremendous basket ice gun queen stream london advanced vessel em bay ain soldiers james palac


Topic #0:
mass dick mail folks seating shone moonlight doctor minded discussed select shouldn beauty believing songs wont plunged dreary entertainment wounds loved sufficed subdued rejoiced desired calculate veins delicate brush undertaking papa pipe riding reported absorbed cliffs masses wicked furiously ordering theirs sack ascertain cheap appeal sheet animal charged sobbing thoughtfully employment returns minister questioned south circle magnificent rouse ends fiercely relatives sorrow hearted urgent favourite limb arrival importance yield fever art tap hush daily trunk stamp impression shell lightly ribs generous operations pardon spy equally sturdy grumbling damp alter lift supported ragged reasons exact echoed band hither frankly ages adventures

Topic #1:
sunk distinguished drifted fish suited guns consisting charming tail subsided peril attempted project unusual powder invalid plate blame prize stunned granted respected lamp amazement larger solid facing bleeding reported repu

### Further resources

[This blog post](https://de.dariah.eu/tatom/feature_selection.html) goes through finding distinctive words using Python in more detail 

Paper: [Fightin’ Words: Lexical Feature Selection and Evaluation for Identifying the Content of Political Conflict](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf), Burt Monroe, Michael Colaresi, Kevin Quinn

[Topic modeling with Textacy](https://github.com/repmax/topic-model/blob/master/topic-modelling.ipynb)